# Random Forest 3

**Zweistufenmodell**

Da unser Datenset extrem unausgewogen ist, sehr viele Werte der zielvariable liegen bei null, nur sehr wenige sind nicht null, kann man versuchen ein Zweistufenmodell anzuwenden.

Man kann erst eine Klassifizierung durchführen, um die Daten in zwei Gruppen zu unterteilen (null vs. nicht-null), und dann ein separates Regressionsmodell auf die nicht-null Daten anwenden.   
Das kann sinnvoll sein, um die Performance deines Modells zu verbessern.

In [1]:
import pandas as pd
import geopandas as gpd

# GeoJSON-Datei laden
gdf = gpd.read_file('../../../data/processed_data/simra_osm_all.geojson')

# Entfernen der Polygone (Geometrie) und nicht benötigter Spalten (type ist ein Feature von SimRa und kann deshlab nicht genutzt werden)
df = gdf.drop(columns=['markers', 'id', 'index_right', 'geometry', 'highway', 'incidents', 'rides', 'type'])


# Anzeigen der ersten Zeilen und Spaltennamen, um zu prüfen, wie die Daten aussehen
df.head()

,score,maxspeed,asphalt,concrete,paving_stone,sett,unpaved,cycleway,footway,highway_rare,living_street,path,primary,residential,secondary,service,tertiary,track
0,0.0,50,0.000000,1.0,0.0,0.0,0.000000,0.250000,0.0,0.0,0.0,0.0,0.0,0.000000,0.750000,0.0,0.000000,0.0
1,0.0,30,1.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,1.000000,0.000000,0.0,0.000000,0.0
2,0.0,50,0.846154,0.0,0.0,0.0,0.153846,0.384615,0.0,0.0,0.0,0.0,0.0,0.153846,0.461538,0.0,0.000000,0.0
3,0.0,30,0.500000,0.0,0.0,0.0,0.500000,0.000000,0.0,0.0,0.0,0.0,0.0,1.000000,0.000000,0.0,0.000000,0.0
4,0.0,50,1.000000,0.0,0.0,0.0,0.000000,0.333333,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.666667,0.0


In [2]:
# Zielvariable (Score) definieren
y = df['score']

In [3]:
# Alle anderen Spalten als Features verwenden ('score')
X = df.drop(columns=['score'])

In [4]:
from sklearn.model_selection import train_test_split

# Aufteilen der Daten in Trainings- und Testsets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Anzahl der Trainingsdaten: {X_train.shape[0]}")
print(f"Anzahl der Testdaten: {X_test.shape[0]}")

Anzahl der Trainingsdaten: 12529
Anzahl der Testdaten: 3133


### Schritt 1: Klassifikationsmodell trainieren

- Klassifikationsmodell erstellen, das vorhersagt, ob ein Wert der Zielvariablen null oder nicht-null ist.

1. **Label-Erstellung:** Erstellen eines Labels, das 1 ist, wenn die Zielvariable nicht null ist, und 0, wenn sie null ist.

In [5]:
y_train_class = (y_train > 0).astype(int)
y_test_class = (y_test > 0).astype(int)


2. **Klassifikationsmodell trainieren:**
   RandomForestClassifier, um die Klassifizierung durchzuführen.

In [6]:
from sklearn.ensemble import RandomForestClassifier

classifier = RandomForestClassifier(random_state=42)
classifier.fit(X_train, y_train_class)


RandomForestClassifier(random_state=42)

3. **Vorhersagen treffen:**
   Verwenden des trainierte Klassifikationsmodells, um vorherzusagen, welche Datenpunkte null und welche nicht-null sind.

In [7]:
y_test_class_pred = classifier.predict(X_test)


### Schritt 2: Regressionsmodell auf nicht-null Werte anwenden

Jetzt, da man weiß, welche Datenpunkte nicht-null sind, kann man ein Regressionsmodell nur auf diese Datenpunkte anwenden.

1. **Nicht-null Daten filtern:** Filtrieren der Trainings- und Testdaten, um nur die nicht-null Werte zu behalten.

In [8]:
X_train_reg = X_train[y_train_class == 1]
y_train_reg = y_train[y_train_class == 1]

X_test_reg = X_test[y_test_class_pred == 1]
y_test_reg = y_test[y_test_class_pred == 1]


2. **Regressionsmodell trainieren:** mit RandomForestRegressor auf den gefilterten Daten trainieren.

In [9]:
from sklearn.ensemble import RandomForestRegressor

regressor = RandomForestRegressor(random_state=42)
regressor.fit(X_train_reg, y_train_reg)


RandomForestRegressor(random_state=42)

3. **Vorhersagen treffen:** Verwenden das trainierten Regressionsmodells, um Vorhersagen für die nicht-null Werte in den Testdaten zu treffen.

In [10]:
y_test_reg_pred = regressor.predict(X_test_reg)


### Schritt 3: Ergebnisse zusammenführen

kombinieren der Ergebnisse beider Modelle, um eine vollständige Vorhersage für den gesamten Testdatensatz zu erhalten.

1. **Erstellen einer Vorhersage für den gesamten Testdatensatz:** Setzen der Vorhersage für die nicht-null Werte in die entsprechenden Positionen.

In [12]:
import numpy as np

y_test_final_pred = np.zeros_like(y_test)
y_test_final_pred[y_test_class_pred == 1] = y_test_reg_pred


2. **Evaluation:** Bewerte die Performance des kombinierten Modells auf dem gesamten Testdatensatz.

In [13]:
from sklearn.metrics import mean_squared_error, r2_score

mse_final = mean_squared_error(y_test, y_test_final_pred)
r2_final = r2_score(y_test, y_test_final_pred)

print(f"Final Test-MSE: {mse_final:.4f}")
print(f"Final Test-R^2: {r2_final:.4f}")


Final Test-MSE: 0.0026
Final Test-R^2: -0.1568


**Interpretation der Ergebnisse:**

- Final Test-MSE (Mean Squared Error): 0.0026
        Was es misst: Der Mean Squared Error (MSE) gibt den durchschnittlichen quadratischen Fehler zwischen den vorhergesagten und den tatsächlichen Werten an.
- Interpretation: Ein MSE von 0.0026 bedeutet, dass die durchschnittliche quadratische Abweichung deiner Vorhersagen von den tatsächlichen Werten relativ klein ist. Da die Zielvariable Werte nahe bei 0 hat, könnte das ein Hinweis darauf sein, dass das Modell die Vorhersagen relativ gut an die tatsächlichen Werte anpasst, aber die kleinen Unterschiede dennoch einen Einfluss haben.

  
    

- Final Test-R² (Bestimmtheitsmaß): -0.1568
        Was es misst: Das R² misst, wie gut die Streuung der Zielvariablen durch das Modell erklärt wird. Ein R² von 1 bedeutet, dass das Modell perfekt erklärt, während ein R² von 0 bedeutet, dass das Modell nicht besser ist als ein einfacher Mittelwert.
- Interpretation: Ein negatives R² (wie -0.1568) deutet darauf hin, dass das Modell schlechter ist als ein Modell, das immer nur den Mittelwert vorhersagt. In anderen Worten, das Modell erfasst die Struktur der Daten nicht gut und liefert weniger zuverlässige Vorhersagen.

### FAZIT
das Zweistufenmodell erbringt keine signifikant besseren Ergebnisse und stellt deshalb keine Alternative als Vorhersagemodell dar